# Aula 14 (05/12/2019)

In [1]:
import numpy as np

## Exercícios de revisão (continuação)

**3.4.** Faça uma função que executa o exercício 3.3 em uma simulação de Monte Carlo com $m$ repetições, armazenando os erros de estimação dos  $m$  modelos ajustados. A função deve retornar os $m$ valores de erros obtidos, sua média e seu desvio-padrão.

In [71]:
# Função de geração de números aleatórios do exercício anterior

def generate_data(f, n = 2, p = 2):
    X = np.random.random((n, p))
    
    y = f(X) + np.random.normal(0, 1, n)
    
    return X, y

# Classe de modelo de regressão do exercício anterior:
class LinearRegression:
    def fit(self,X,y):
        self.coef = np.linalg.inv(X.transpose() @ X) @ X.transpose() @ y
        return self
    
    def predict(self, X):
        return X @ self.coef
    
    def score(self, X, y):
        return np.mean((y - self.predict(X))**2)

In [72]:
# Argumentos:
# m: número de repetições
# n: tamanho dos conjuntos de cada repetição

def LR_MC(m, n):
    erros = np.zeros(m)
    
    f = lambda x: 2*x[:,0]**2 + 3*x[:,1] + 1
    
    for i in range(m):
        # Treinamos nosso modelo de regressão linear em um conjunto de dados.
        X_train, y_train = generate_data(f,n,2)
        LR_train = LinearRegression().fit(X_train,y_train)
        
        # Estamos interessados no erro obtido na comparação de nosso modelo
        # de regressão em um outro conjunto de dados
        X_test, y_test = generate_data(f,n,2)
        erros[i] = LR_train.score(X_test,y_test)
        
    return erros, np.mean(erros), np.std(erros)

In [75]:
np.random.seed(5122019)

resultados, media, desvio = LR_MC(10, 5)

print("Erros: ",resultados)
print()
print("Média dos erros:",media)
print()
print("Desvio padrão dos erros:", desvio)

Erros:  [1.73990227 0.15263589 2.22568124 1.1162401  0.48513254 1.70970077
 0.81033775 1.10603743 1.97437938 0.86900679]

Média dos erros: 1.2189054178959515

Desvio padrão dos erros: 0.6391123176410289


**3.5.** Faça uma função que aproxima a integral de uma função qualquer em um intervalo $[a,b]$, usando a [regra de Simpson](https://pt.wikipedia.org/wiki/Integra%C3%A7%C3%A3o_num%C3%A9rica). Use essa função em uma outra função que deve calcular  $P(X < x_{i})$, para  $i=1,…,n$, onde  $X∼N(\mu,\sigma^{2})$.

**Regra de Simpson:**

$\large \int_{a}^{b}$ $f(x)dx \approx (b - a)$$\large \frac{f(a) + 4 f(\frac{a + b}{2}) + f(b)}{6}$

In [77]:
# Pela definição da integral de Simpson:

def integral_simpson(f, a, b):
    return (b - a)*(f(a) + 4*f((a + b)/2) + f(b))/6

Testando para  $f_{1}(x) = x^{4}$  em  $[0,1]$ e  $f_{2}(x) = sen(x)$  em  $[0, \pi]$ (analiticamente, essas integrais tem valor $\large{\frac{1}{5}}$  e  $2$ ): 

In [78]:
f1 = lambda x: x**4
f2 = lambda x: np.sin(x)

integral_simpson(f1, 0, 1), integral_simpson(f2, 0, np.pi)

(0.20833333333333334, 2.0943951023931953)

Para resultados mais precisos, podemos dividir o intervalo $[a,b]$ em $n$ subintervalos:

In [81]:
def integral_simpson_n(f, a, b, n = 101):
    
    def integral_simpson(f, a, b):
        return (b - a)*(f(a) + 4*f((a + b)/2) + f(b))/6
    
    x = np.linspace(a, b, n)
    
    integral = np.zeros(n -1)
    
    for i in range(n - 1):
        integral[i] = integral_simpson(f, x[i], x[i + 1])
        
    return sum(integral)

Testando nos mesmos exemplos de antes, espera-se resultados de precisão semelhante ou maior:

In [82]:
integral_simpson_n(f1, 0, 1), integral_simpson_n(f2, 0, np.pi)

(0.20000000008333332, 2.0000000006764718)

Podemos usar a função da intergral de Simpson para calcular a função de distribuição acumulada (intervalo $(-\infty,x]$ ) de uma v.a. normal.

Não podemos integrar de  $-\infty$  até  $x$ , mas temos que a prob acumulada de $-\infty$  até  $\mu$  é $0.5$ . Logo, podemos simplesmente aproximar a integral entre  $\mu$  e  $x$  e somar (ou subtrair) esse valor com $0.5$ 

In [83]:
def normal_acumulada(x, mu = 0, var = 1):
    
    # Densidade da normal
    f = lambda x: (np.sqrt(2*np.pi*var)**(-1))*np.exp(-((x - mu)**2)/(2*var))
    
    if x > mu:
        prob = 0.5 + integral_simpson_n(f, mu, x)
    else:
        prob = 0.5 - integral_simpson_n(f, x, mu)
    return prob

In [85]:
normal_acumulada(1.96)

0.975002104846839